In [1]:
# import modules
import panel as pn
pn.extension('tabulator')
import pandas as pd
import numpy as np
from panel.template import FastListTemplate
from pathlib import Path
#from yahoo_fin.stock_info import get_data
import datetime
from matplotlib.figure import Figure
from matplotlib import cm
%matplotlib inline
import hvplot.pandas
import holoviews as hv
from holoviews import opts


import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

# import modules that help build tabs
#import modules.helpers as helpers
#import modules.HistoricalData as hst
#import modules.MCTab as MCTab
#import modules.intro as intro
#import modules.profile as prf
#import modules.algorithmic_functions as af


#import pandas_ta as ta
#import yfinance as yf
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.preprocessing import StandardScaler

#from pandas.tseries.offsets import DateOffset
#from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, roc_auc_score, f1_score
#import seaborn as sns

#from joblib import dump, load

2023-04-11 16:35:23.045233: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Load Data

## *Data has been preprocessed*

* If necessary/desired, use 'build_portfolio_signal_ml_df' to pull machine learning data to create/refresht he test/train datasets
* Load the test/train datasets
* uncomment the below code in order to create or refresh the test/train datasets

In [2]:
# load X_train_full and X_test_full
X_train_full_conservative = pd.read_csv(Path("./data/X_train_full_conservative.csv"), index_col="Unnamed: 0", parse_dates=True, infer_datetime_format=True)
X_test_full_conservative = pd.read_csv(Path("./data/X_test_full_conservative.csv"), index_col="Unnamed: 0", parse_dates=True, infer_datetime_format=True)

X_train_full_balanced = pd.read_csv(Path("./data/X_train_full_balanced.csv"), index_col="Unnamed: 0", parse_dates=True, infer_datetime_format=True)
X_test_full_balanced = pd.read_csv(Path("./data/X_test_full_balanced.csv"), index_col="Unnamed: 0", parse_dates=True, infer_datetime_format=True)

X_train_full_growth = pd.read_csv(Path("./data/X_train_full_growth.csv"), index_col="Unnamed: 0", parse_dates=True, infer_datetime_format=True)
X_test_full_growth = pd.read_csv(Path("./data/X_test_full_growth.csv"), index_col="Unnamed: 0", parse_dates=True, infer_datetime_format=True)

X_train_full_aggressive = pd.read_csv(Path("./data/X_train_full_aggressive.csv"), index_col="Unnamed: 0", parse_dates=True, infer_datetime_format=True)
X_test_full_aggressive = pd.read_csv(Path("./data/X_test_full_aggressive.csv"), index_col="Unnamed: 0", parse_dates=True, infer_datetime_format=True)

X_train_full_alternative = pd.read_csv(Path("./data/X_train_full_alternative.csv"), index_col="Unnamed: 0", parse_dates=True, infer_datetime_format=True)
X_test_full_alternative = pd.read_csv(Path("./data/X_test_full_alternative.csv"), index_col="Unnamed: 0", parse_dates=True, infer_datetime_format=True)



#load y_train and y_test
y_train_conservative = pd.read_csv(Path("./data/y_train_conservative.csv"), index_col="Unnamed: 0", parse_dates=True, infer_datetime_format=True)
y_test_conservative = pd.read_csv(Path("./data/y_test_conservative.csv"), index_col="Unnamed: 0", parse_dates=True, infer_datetime_format=True)

y_train_balanced = pd.read_csv(Path("./data/y_train_balanced.csv"), index_col="Unnamed: 0", parse_dates=True, infer_datetime_format=True)
y_test_balanced = pd.read_csv(Path("./data/y_test_balanced.csv"), index_col="Unnamed: 0", parse_dates=True, infer_datetime_format=True)

y_train_growth = pd.read_csv(Path("./data/y_train_growth.csv"), index_col="Unnamed: 0", parse_dates=True, infer_datetime_format=True)
y_test_growth = pd.read_csv(Path("./data/y_test_growth.csv"), index_col="Unnamed: 0", parse_dates=True, infer_datetime_format=True)

y_train_aggressive = pd.read_csv(Path("./data/y_train_aggressive.csv"), index_col="Unnamed: 0", parse_dates=True, infer_datetime_format=True)
y_test_aggressive = pd.read_csv(Path("./data/y_test_aggressive.csv"), index_col="Unnamed: 0", parse_dates=True, infer_datetime_format=True)

y_train_alternative = pd.read_csv(Path("./data/y_train_alternative.csv"), index_col="Unnamed: 0", parse_dates=True, infer_datetime_format=True)
y_test_alternative = pd.read_csv(Path("./data/y_test_alternative.csv"), index_col="Unnamed: 0", parse_dates=True, infer_datetime_format=True)

datafiles = {'conservative': [X_train_full_conservative,
                              X_test_full_conservative, 
                              y_train_conservative, 
                              y_test_conservative],
            'balanced': [X_train_full_balanced,
                              X_test_full_balanced, 
                              y_train_balanced, 
                              y_test_balanced],
            'growth': [X_train_full_growth,
                              X_test_full_growth, 
                              y_train_growth, 
                              y_test_growth],
            'aggressive': [X_train_full_aggressive,
                              X_test_full_aggressive, 
                              y_train_aggressive, 
                              y_test_aggressive],
            'alternative': [X_train_full_alternative,
                              X_test_full_alternative, 
                              y_train_alternative, 
                              y_test_alternative]}

portfolios = ['conservative', 'balanced', 'growth', 'aggressive','alternative']

# Create Model

## Use best parameters from tuning notebook to train one model per portfolio class for both the full and reduced feature sets

### Full Feature Set

In [3]:
# Create the scaler instance
X_scaler = StandardScaler()

# Fit the scaler
X_scaler.fit(X_train_full_conservative)

# Scale the data
X_train_full_conservative_scaled = X_scaler.transform(X_train_full_conservative)
X_test_full_conservative_scaled = X_scaler.transform(X_test_full_conservative)




## *Conservative Portfolio*

In [4]:
number_input_features = 18
hidden_nodes_layer1 = 18
hidden_nodes_layer2 = 3
activation_1 = 'relu'
activation_2 = 'relu'
lr = 0.001

# Create a sequential neural network model
nn_conservative = Sequential()

# Add the first hidden layer
nn_conservative.add(Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation=activation_1))

# Add the second hidden layer
nn_conservative.add(Dense(units=hidden_nodes_layer2, activation=activation_2))

# Add the output layer
nn_conservative.add(Dense(units=1, activation="sigmoid"))


# Compile the model 
# Set the parameters as mean_squared_error, adam, and accuracy.
nn_conservative.compile(loss="binary_crossentropy", optimizer=tf.keras.optimizers.Adam(learning_rate=lr), metrics=["accuracy"])


# Fit the model
deep_net_conservative_model = nn_conservative.fit(X_train_full_conservative_scaled, y_train_conservative, epochs=100, verbose=0)

# Evaluate Conservative Model using testing data
model_loss, model_accuracy = nn_conservative.evaluate(X_test_full_conservative_scaled, y_test_conservative, verbose=2)

print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")


2023-04-11 16:35:29.461650: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


7/7 - 0s - loss: 0.7235 - accuracy: 0.5000 - 123ms/epoch - 18ms/step
Loss: 0.7234643697738647, Accuracy: 0.5


## *Balanced Portfolio*

In [5]:
# Create the scaler instance
X_scaler = StandardScaler()

# Fit the scaler
X_scaler.fit(X_train_full_balanced)

# Scale the data
X_train_full_balanced_scaled = X_scaler.transform(X_train_full_balanced)
X_test_full_balanced_scaled = X_scaler.transform(X_test_full_balanced)

number_input_features = 18
hidden_nodes_layer1 = 14
hidden_nodes_layer2 = 3
activation_1 = 'relu'
activation_2 = 'tanh'
lr = 0.001

# Create a sequential neural network model
nn_balanced = Sequential()

# Add the first hidden layer
nn_balanced.add(Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation=activation_1))

# Add the second hidden layer
nn_balanced.add(Dense(units=hidden_nodes_layer2, activation=activation_2))

# Add the output layer
nn_balanced.add(Dense(units=1, activation="sigmoid"))

# Compile the model 
# Set the parameters as mean_squared_error, adam, and accuracy.
nn_balanced.compile(loss="binary_crossentropy", optimizer=tf.keras.optimizers.Adam(learning_rate=lr), metrics=["accuracy"])

# Fit the model
deep_net_balanced_model = nn_balanced.fit(X_train_full_balanced_scaled, y_train_balanced, epochs=100, verbose=0)

# Evaluate Balanced Model using testing data
model_loss, model_accuracy = nn_balanced.evaluate(X_test_full_balanced_scaled, y_test_balanced, verbose=2)

print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")


7/7 - 0s - loss: 0.7248 - accuracy: 0.4811 - 100ms/epoch - 14ms/step
Loss: 0.7248082160949707, Accuracy: 0.4811320900917053


## *Growth Portfolio*

In [6]:
# Create the scaler instance
X_scaler = StandardScaler()

# Fit the scaler
X_scaler.fit(X_train_full_growth)

# Scale the data
X_train_full_growth_scaled = X_scaler.transform(X_train_full_growth)
X_test_full_growth_scaled = X_scaler.transform(X_test_full_growth)

number_input_features = 18
hidden_nodes_layer1 = 20
hidden_nodes_layer2 = 3
activation_1 = 'tanh'
activation_2 = 'relu'
lr = 0.01

# Create a sequential neural network model
nn_growth = Sequential()

# Add the first hidden layer
nn_growth.add(Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation=activation_1))

# Add the second hidden layer
nn_growth.add(Dense(units=hidden_nodes_layer2, activation=activation_2))

# Add the output layer
nn_growth.add(Dense(units=1, activation="sigmoid"))

# Compile the model 
# Set the parameters as mean_squared_error, adam, and accuracy.
nn_growth.compile(loss="binary_crossentropy", optimizer=tf.keras.optimizers.Adam(learning_rate=lr), metrics=["accuracy"])

# Fit the model
deep_net_growth_model = nn_growth.fit(X_train_full_growth_scaled, y_train_growth, epochs=100, verbose=0)

# Evaluate Growth Model using testing data
model_loss, model_accuracy = nn_growth.evaluate(X_test_full_growth_scaled, y_test_growth, verbose=2)

print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")


12/12 - 0s - loss: 1.4353 - accuracy: 0.4744 - 99ms/epoch - 8ms/step
Loss: 1.4352911710739136, Accuracy: 0.4743935167789459


## *Aggressive Portfolio*

In [7]:
# Create the scaler instance
X_scaler = StandardScaler()

# Fit the scaler
X_scaler.fit(X_train_full_aggressive)

# Scale the data
X_train_full_aggressive_scaled = X_scaler.transform(X_train_full_aggressive)
X_test_full_aggressive_scaled = X_scaler.transform(X_test_full_aggressive)

number_input_features = 18
hidden_nodes_layer1 = 28
hidden_nodes_layer2 = 3
activation_1 = 'relu'
activation_2 = 'tanh'
lr = 0.01

# Create a sequential neural network model
nn_aggressive = Sequential()

# Add the first hidden layer
nn_aggressive.add(Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation=activation_1))

# Add the second hidden layer
nn_aggressive.add(Dense(units=hidden_nodes_layer2, activation=activation_2))

# Add the output layer
nn_aggressive.add(Dense(units=1, activation="sigmoid"))

# Compile the model 
# Set the parameters as mean_squared_error, adam, and accuracy.
nn_aggressive.compile(loss="binary_crossentropy", optimizer=tf.keras.optimizers.Adam(learning_rate=lr), metrics=["accuracy"])

# Fit the model
deep_net_aggressive_model = nn_aggressive.fit(X_train_full_aggressive_scaled, y_train_aggressive, epochs=100, verbose=0)

model_loss, model_accuracy = nn_aggressive.evaluate(X_test_full_aggressive_scaled, y_test_aggressive, verbose=2)

print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

12/12 - 0s - loss: 0.8035 - accuracy: 0.5364 - 103ms/epoch - 9ms/step
Loss: 0.8035091757774353, Accuracy: 0.5363881587982178


## Alternative Portfolio

In [8]:
# Create the scaler instance
X_scaler = StandardScaler()

# Fit the scaler
X_scaler.fit(X_train_full_alternative)

# Scale the data
X_train_full_alternative_scaled = X_scaler.transform(X_train_full_alternative)
X_test_full_alternative_scaled = X_scaler.transform(X_test_full_alternative)

In [9]:
number_input_features = 18
hidden_nodes_layer1 = 14
hidden_nodes_layer2 = 3
activation_1 = 'tanh'
activation_2 = 'tanh'
lr = 0.01

# Create a sequential neural network model
nn_alternative = Sequential()

# Add the first hidden layer
nn_alternative.add(Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation=activation_1))

# Add the second hidden layer
nn_alternative.add(Dense(units=hidden_nodes_layer2, activation=activation_2))

# Add the output layer
nn_alternative.add(Dense(units=1, activation="sigmoid"))

# Compile the model 
# Set the parameters as mean_squared_error, adam, and accuracy.
nn_alternative.compile(loss="binary_crossentropy", optimizer=tf.keras.optimizers.Adam(learning_rate=lr), metrics=["accuracy"])

# Fit the model
deep_net_alternative_model = nn_alternative.fit(X_train_full_alternative_scaled, y_train_alternative, epochs=100, verbose=0)

model_loss, model_accuracy = nn_alternative.evaluate(X_test_full_alternative_scaled, y_test_alternative, verbose=2)

print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

12/12 - 0s - loss: 1.0249 - accuracy: 0.5040 - 105ms/epoch - 9ms/step
Loss: 1.0248531103134155, Accuracy: 0.5040431022644043


In [10]:
#Conservative Portfolio
# deep_net_conservative_model = nn_conservative.fit(X_train_full_conservative_scaled, y_train_conservative, epochs=100)

# Evaluate Conservative Model using testing data
model_loss, model_accuracy = nn_conservative.evaluate(X_test_full_conservative_scaled, y_test_conservative, verbose=0)

print(f"\nConservative Model\nLoss: {model_loss}, Accuracy: {model_accuracy}")


#Balanced Portfolio
# deep_net_balanced_model = nn_balanced.fit(X_train_full_balanced_scaled, y_train_balanced, epochs=100)

#Evaluate Balanced Model using testing data
model_loss, model_accuracy = nn_balanced.evaluate(X_test_full_balanced_scaled, y_test_balanced, verbose=0)

print(f"\nBalanced Model\nLoss: {model_loss}, Accuracy: {model_accuracy}")

#Growth Portfolio
# deep_net_growth_model = nn_growth.fit(X_train_full_growth_scaled, y_train_growth, epochs=100)

#Evaluate Growth Model using testing data
model_loss, model_accuracy = nn_growth.evaluate(X_test_full_growth_scaled, y_test_growth, verbose=0)

print(f"\nGrowth Model\nLoss: {model_loss}, Accuracy: {model_accuracy}")


#Aggressive Portfolio
# deep_net_aggressive_model = nn_aggressive.fit(X_train_full_aggressive_scaled, y_train_aggressive, epochs=100)

#Evaluate Aggressive Model using testing data
model_loss, model_accuracy = nn_aggressive.evaluate(X_test_full_aggressive_scaled, y_test_aggressive, verbose=0)

print(f"\nAggressive Model\nLoss: {model_loss}, Accuracy: {model_accuracy}")

#Alternative Portfolio
# deep_net_alternative_model = nn_alternative.fit(X_train_full_alternative_scaled, y_train_alternative, epochs=100)

#Evaluate Alternative Model using testing data
model_loss, model_accuracy = nn_alternative.evaluate(X_test_full_alternative_scaled, y_test_alternative, verbose=0)

print(f"\nAlternative Model\nLoss: {model_loss}, Accuracy: {model_accuracy}")





Conservative Model
Loss: 0.7234643697738647, Accuracy: 0.5

Balanced Model
Loss: 0.7248082160949707, Accuracy: 0.4811320900917053

Growth Model
Loss: 1.4352911710739136, Accuracy: 0.4743935167789459

Aggressive Model
Loss: 0.8035091757774353, Accuracy: 0.5363881587982178

Alternative Model
Loss: 1.0248531103134155, Accuracy: 0.5040431022644043


In [11]:
full_alternative_proba = nn_alternative.predict(X_test_full_alternative_scaled)
full_aggressive_proba = nn_aggressive.predict(X_test_full_aggressive_scaled)
full_growth_proba = nn_growth.predict(X_test_full_growth_scaled)
full_balanced_proba = nn_balanced.predict(X_test_full_balanced_scaled)
full_conservative_proba = nn_conservative.predict(X_test_full_conservative_scaled)

full_alternative_preds = tf.cast(full_alternative_proba >= 0.5, tf.int32)
full_aggressive_preds = tf.cast(full_aggressive_proba >= 0.5, tf.int32)
full_growth_preds = tf.cast(full_growth_proba >= 0.5, tf.int32)
full_balanced_preds = tf.cast(full_balanced_proba >= 0.5, tf.int32)
full_conservative_preds = tf.cast(full_conservative_proba >= 0.5, tf.int32)

full_preds = {'alternative': [full_alternative_proba, full_alternative_preds],
              'aggressive': [full_aggressive_proba, full_aggressive_preds],
              'growth': [full_growth_proba, full_growth_preds],
              'balanced': [full_balanced_proba, full_balanced_preds],
              'conservative': [full_conservative_proba, full_conservative_preds]
             }

7/7 [==============================] - 0s 1ms/step


In [12]:
full_results = pd.DataFrame()
for p in portfolios:
    y_test = datafiles[p][3]
    proba = full_preds[p][0]
    preds = full_preds[p][1]
    roc = roc_auc_score(y_test,proba)
    f1 = f1_score(y_test, preds)
    full_results.loc[p, 'roc'] = roc
    full_results.loc[p, 'f1'] = f1
full_results['type'] = 'full'

In [13]:
full_results

,roc,f1,type
conservative,0.473393,0.442105,full
balanced,0.512834,0.466019,full
growth,0.514452,0.615385,full
aggressive,0.545075,0.530055,full
alternative,0.519740,0.619835,full


### Reduced Feature Set

In [14]:
# load X_train_reduced and X_test_reduced
X_train_reduced_conservative = pd.read_csv(Path("./data/X_train_reduced_conservative.csv"), index_col="Unnamed: 0", parse_dates=True, infer_datetime_format=True)
X_test_reduced_conservative = pd.read_csv(Path("./data/X_test_reduced_conservative.csv"), index_col="Unnamed: 0", parse_dates=True, infer_datetime_format=True)

X_train_reduced_balanced = pd.read_csv(Path("./data/X_train_reduced_balanced.csv"), index_col="Unnamed: 0", parse_dates=True, infer_datetime_format=True)
X_test_reduced_balanced = pd.read_csv(Path("./data/X_test_reduced_balanced.csv"), index_col="Unnamed: 0", parse_dates=True, infer_datetime_format=True)

X_train_reduced_growth = pd.read_csv(Path("./data/X_train_reduced_growth.csv"), index_col="Unnamed: 0", parse_dates=True, infer_datetime_format=True)
X_test_reduced_growth = pd.read_csv(Path("./data/X_test_reduced_growth.csv"), index_col="Unnamed: 0", parse_dates=True, infer_datetime_format=True)

X_train_reduced_aggressive = pd.read_csv(Path("./data/X_train_reduced_aggressive.csv"), index_col="Unnamed: 0", parse_dates=True, infer_datetime_format=True)
X_test_reduced_aggressive = pd.read_csv(Path("./data/X_test_reduced_aggressive.csv"), index_col="Unnamed: 0", parse_dates=True, infer_datetime_format=True)

X_train_reduced_alternative = pd.read_csv(Path("./data/X_train_reduced_alternative.csv"), index_col="Unnamed: 0", parse_dates=True, infer_datetime_format=True)
X_test_reduced_alternative = pd.read_csv(Path("./data/X_test_reduced_alternative.csv"), index_col="Unnamed: 0", parse_dates=True, infer_datetime_format=True)



#load y_train and y_test
y_train_conservative = pd.read_csv(Path("./data/y_train_conservative.csv"), index_col="Unnamed: 0", parse_dates=True, infer_datetime_format=True)
y_test_conservative = pd.read_csv(Path("./data/y_test_conservative.csv"), index_col="Unnamed: 0", parse_dates=True, infer_datetime_format=True)

y_train_balanced = pd.read_csv(Path("./data/y_train_balanced.csv"), index_col="Unnamed: 0", parse_dates=True, infer_datetime_format=True)
y_test_balanced = pd.read_csv(Path("./data/y_test_balanced.csv"), index_col="Unnamed: 0", parse_dates=True, infer_datetime_format=True)

y_train_growth = pd.read_csv(Path("./data/y_train_growth.csv"), index_col="Unnamed: 0", parse_dates=True, infer_datetime_format=True)
y_test_growth = pd.read_csv(Path("./data/y_test_growth.csv"), index_col="Unnamed: 0", parse_dates=True, infer_datetime_format=True)

y_train_aggressive = pd.read_csv(Path("./data/y_train_aggressive.csv"), index_col="Unnamed: 0", parse_dates=True, infer_datetime_format=True)
y_test_aggressive = pd.read_csv(Path("./data/y_test_aggressive.csv"), index_col="Unnamed: 0", parse_dates=True, infer_datetime_format=True)

y_train_alternative = pd.read_csv(Path("./data/y_train_alternative.csv"), index_col="Unnamed: 0", parse_dates=True, infer_datetime_format=True)
y_test_alternative = pd.read_csv(Path("./data/y_test_alternative.csv"), index_col="Unnamed: 0", parse_dates=True, infer_datetime_format=True)

datafiles = {'conservative': [X_train_reduced_conservative,
                              X_test_reduced_conservative, 
                              y_train_conservative, 
                              y_test_conservative],
            'balanced': [X_train_reduced_balanced,
                              X_test_reduced_balanced, 
                              y_train_balanced, 
                              y_test_balanced],
            'growth': [X_train_reduced_growth,
                              X_test_reduced_growth, 
                              y_train_growth, 
                              y_test_growth],
            'aggressive': [X_train_reduced_aggressive,
                              X_test_reduced_aggressive, 
                              y_train_aggressive, 
                              y_test_aggressive],
            'alternative': [X_train_reduced_alternative,
                              X_test_reduced_alternative, 
                              y_train_alternative, 
                              y_test_alternative]}

portfolios = ['conservative', 'balanced', 'growth', 'aggressive','alternative']

In [15]:
# Create the scaler instance
X_scaler = StandardScaler()

# Fit the scaler
X_scaler.fit(X_train_reduced_conservative)

# Scale the data
X_train_reduced_conservative_scaled = X_scaler.transform(X_train_reduced_conservative)
X_test_reduced_conservative_scaled = X_scaler.transform(X_test_reduced_conservative)




## *Conservative Portfolio*

In [16]:
number_input_features = 7
hidden_nodes_layer1 = 18
hidden_nodes_layer2 = 9
activation_1 = 'tanh'
activation_2 = 'relu'
lr = 0.01

# Create a sequential neural network model
nn_conservative = Sequential()

# Add the first hidden layer
nn_conservative.add(Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation=activation_1))

# Add the second hidden layer
nn_conservative.add(Dense(units=hidden_nodes_layer2, activation=activation_2))

# Add the output layer
nn_conservative.add(Dense(units=1, activation="sigmoid"))


# Compile the model 
# Set the parameters as mean_squared_error, adam, and accuracy.
nn_conservative.compile(loss="binary_crossentropy", optimizer=tf.keras.optimizers.Adam(learning_rate=lr), metrics=["accuracy"])


# Fit the model
deep_net_conservative_model = nn_conservative.fit(X_train_reduced_conservative_scaled, y_train_conservative, epochs=100, verbose=0)

# Evaluate Conservative Model using testing data
model_loss, model_accuracy = nn_conservative.evaluate(X_test_reduced_conservative_scaled, y_test_conservative, verbose=2)

print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")


7/7 - 0s - loss: 0.7923 - accuracy: 0.5189 - 102ms/epoch - 15ms/step
Loss: 0.7923059463500977, Accuracy: 0.5188679099082947


## *Balanced Portfolio*

In [17]:
# Create the scaler instance
X_scaler = StandardScaler()

# Fit the scaler
X_scaler.fit(X_train_reduced_balanced)

# Scale the data
X_train_reduced_balanced_scaled = X_scaler.transform(X_train_reduced_balanced)
X_test_reduced_balanced_scaled = X_scaler.transform(X_test_reduced_balanced)

number_input_features = 7
hidden_nodes_layer1 = 12
hidden_nodes_layer2 = 7
activation_1 = 'relu'
activation_2 = 'relu'
lr = 0.001

# Create a sequential neural network model
nn_balanced = Sequential()

# Add the first hidden layer
nn_balanced.add(Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation=activation_1))

# Add the second hidden layer
nn_balanced.add(Dense(units=hidden_nodes_layer2, activation=activation_2))

# Add the output layer
nn_balanced.add(Dense(units=1, activation="sigmoid"))

# Compile the model 
# Set the parameters as mean_squared_error, adam, and accuracy.
nn_balanced.compile(loss="binary_crossentropy", optimizer=tf.keras.optimizers.Adam(learning_rate=lr), metrics=["accuracy"])

# Fit the model
deep_net_balanced_model = nn_balanced.fit(X_train_reduced_balanced_scaled, y_train_balanced, epochs=100, verbose=0)

# Evaluate Balanced Model using testing data
model_loss, model_accuracy = nn_balanced.evaluate(X_test_reduced_balanced_scaled, y_test_balanced, verbose=2)

print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")


7/7 - 0s - loss: 0.7013 - accuracy: 0.5094 - 101ms/epoch - 14ms/step
Loss: 0.7012689709663391, Accuracy: 0.5094339847564697


## *Growth Portfolio*

In [18]:
# Create the scaler instance
X_scaler = StandardScaler()

# Fit the scaler
X_scaler.fit(X_train_reduced_growth)

# Scale the data
X_train_reduced_growth_scaled = X_scaler.transform(X_train_reduced_growth)
X_test_reduced_growth_scaled = X_scaler.transform(X_test_reduced_growth)

number_input_features = 7
hidden_nodes_layer1 = 36
hidden_nodes_layer2 = 9
activation_1 = 'relu'
activation_2 = 'relu'
lr = 0.0001

# Create a sequential neural network model
nn_growth = Sequential()

# Add the first hidden layer
nn_growth.add(Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation=activation_1))

# Add the second hidden layer
nn_growth.add(Dense(units=hidden_nodes_layer2, activation=activation_2))

# Add the output layer
nn_growth.add(Dense(units=1, activation="sigmoid"))

# Compile the model 
# Set the parameters as mean_squared_error, adam, and accuracy.
nn_growth.compile(loss="binary_crossentropy", optimizer=tf.keras.optimizers.Adam(learning_rate=lr), metrics=["accuracy"])

# Fit the model
deep_net_growth_model = nn_growth.fit(X_train_reduced_growth_scaled, y_train_growth, epochs=100, verbose=0)

# Evaluate Growth Model using testing data
model_loss, model_accuracy = nn_growth.evaluate(X_test_reduced_growth_scaled, y_test_growth, verbose=2)

print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")


12/12 - 0s - loss: 0.6864 - accuracy: 0.5418 - 106ms/epoch - 9ms/step
Loss: 0.6863937377929688, Accuracy: 0.5417789816856384


## *Aggressive Portfolio*

In [19]:
# Create the scaler instance
X_scaler = StandardScaler()

# Fit the scaler
X_scaler.fit(X_train_reduced_aggressive)

# Scale the data
X_train_reduced_aggressive_scaled = X_scaler.transform(X_train_reduced_aggressive)
X_test_reduced_aggressive_scaled = X_scaler.transform(X_test_reduced_aggressive)

number_input_features = 7
hidden_nodes_layer1 = 30
hidden_nodes_layer2 = 3
activation_1 = 'tanh'
activation_2 = 'relu'
lr = 0.01

# Create a sequential neural network model
nn_aggressive = Sequential()

# Add the first hidden layer
nn_aggressive.add(Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation=activation_1))

# Add the second hidden layer
nn_aggressive.add(Dense(units=hidden_nodes_layer2, activation=activation_2))

# Add the output layer
nn_aggressive.add(Dense(units=1, activation="sigmoid"))

# Compile the model 
# Set the parameters as mean_squared_error, adam, and accuracy.
nn_aggressive.compile(loss="binary_crossentropy", optimizer=tf.keras.optimizers.Adam(learning_rate=lr), metrics=["accuracy"])

# Fit the model
deep_net_aggressive_model = nn_aggressive.fit(X_train_reduced_aggressive_scaled, y_train_aggressive, epochs=100, verbose=0)

model_loss, model_accuracy = nn_aggressive.evaluate(X_test_reduced_aggressive_scaled, y_test_aggressive, verbose=2)

print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

12/12 - 0s - loss: 0.9432 - accuracy: 0.4960 - 157ms/epoch - 13ms/step
Loss: 0.9432427883148193, Accuracy: 0.4959568679332733


## Alternative Portfolio

In [20]:
# Create the scaler instance
X_scaler = StandardScaler()

# Fit the scaler
X_scaler.fit(X_train_reduced_alternative)

# Scale the data
X_train_reduced_alternative_scaled = X_scaler.transform(X_train_reduced_alternative)
X_test_reduced_alternative_scaled = X_scaler.transform(X_test_reduced_alternative)

In [21]:
number_input_features = 7
hidden_nodes_layer1 = 8
hidden_nodes_layer2 = 3
activation_1 = 'relu'
activation_2 = 'relu'
lr = 0.001

# Create a sequential neural network model
nn_alternative = Sequential()

# Add the first hidden layer
nn_alternative.add(Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation=activation_1))

# Add the second hidden layer
nn_alternative.add(Dense(units=hidden_nodes_layer2, activation=activation_2))

# Add the output layer
nn_alternative.add(Dense(units=1, activation="sigmoid"))

# Compile the model 
# Set the parameters as mean_squared_error, adam, and accuracy.
nn_alternative.compile(loss="binary_crossentropy", optimizer=tf.keras.optimizers.Adam(learning_rate=lr), metrics=["accuracy"])

# Fit the model
deep_net_alternative_model = nn_alternative.fit(X_train_reduced_alternative_scaled, y_train_alternative, epochs=100, verbose=0)

model_loss, model_accuracy = nn_alternative.evaluate(X_test_reduced_alternative_scaled, y_test_alternative, verbose=2)

print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

12/12 - 0s - loss: 0.6959 - accuracy: 0.4906 - 104ms/epoch - 9ms/step
Loss: 0.6959148645401001, Accuracy: 0.49056604504585266


In [22]:
#Conservative Portfolio
# deep_net_conservative_model = nn_conservative.fit(X_train_full_conservative_scaled, y_train_conservative, epochs=100)

# Evaluate Conservative Model using testing data
model_loss, model_accuracy = nn_conservative.evaluate(X_test_reduced_conservative_scaled, y_test_conservative, verbose=0)

print(f"\nConservative Model\nLoss: {model_loss}, Accuracy: {model_accuracy}")


#Balanced Portfolio
# deep_net_balanced_model = nn_balanced.fit(X_train_full_balanced_scaled, y_train_balanced, epochs=100)

#Evaluate Balanced Model using testing data
model_loss, model_accuracy = nn_balanced.evaluate(X_test_reduced_balanced_scaled, y_test_balanced, verbose=0)

print(f"\nBalanced Model\nLoss: {model_loss}, Accuracy: {model_accuracy}")

#Growth Portfolio
# deep_net_growth_model = nn_growth.fit(X_train_full_growth_scaled, y_train_growth, epochs=100)

#Evaluate Growth Model using testing data
model_loss, model_accuracy = nn_growth.evaluate(X_test_reduced_growth_scaled, y_test_growth, verbose=0)

print(f"\nGrowth Model\nLoss: {model_loss}, Accuracy: {model_accuracy}")


#Aggressive Portfolio
# deep_net_aggressive_model = nn_aggressive.fit(X_train_full_aggressive_scaled, y_train_aggressive, epochs=100)

#Evaluate Aggressive Model using testing data
model_loss, model_accuracy = nn_aggressive.evaluate(X_test_reduced_aggressive_scaled, y_test_aggressive, verbose=0)

print(f"\nAggressive Model\nLoss: {model_loss}, Accuracy: {model_accuracy}")

#Alternative Portfolio
# deep_net_alternative_model = nn_alternative.fit(X_train_full_alternative_scaled, y_train_alternative, epochs=100)

#Evaluate Alternative Model using testing data
model_loss, model_accuracy = nn_alternative.evaluate(X_test_reduced_alternative_scaled, y_test_alternative, verbose=0)

print(f"\nAlternative Model\nLoss: {model_loss}, Accuracy: {model_accuracy}")





Conservative Model
Loss: 0.7923059463500977, Accuracy: 0.5188679099082947

Balanced Model
Loss: 0.7012689709663391, Accuracy: 0.5094339847564697

Growth Model
Loss: 0.6863937377929688, Accuracy: 0.5417789816856384

Aggressive Model
Loss: 0.9432427883148193, Accuracy: 0.4959568679332733

Alternative Model
Loss: 0.6959148645401001, Accuracy: 0.49056604504585266


In [23]:
reduced_alternative_proba = nn_alternative.predict(X_test_reduced_alternative_scaled)
reduced_aggressive_proba = nn_aggressive.predict(X_test_reduced_aggressive_scaled)
reduced_growth_proba = nn_growth.predict(X_test_reduced_growth_scaled)
reduced_balanced_proba = nn_balanced.predict(X_test_reduced_balanced_scaled)
reduced_conservative_proba = nn_conservative.predict(X_test_reduced_conservative_scaled)

reduced_alternative_preds = tf.cast(reduced_alternative_proba >= 0.5, tf.int32)
reduced_aggressive_preds = tf.cast(reduced_aggressive_proba >= 0.5, tf.int32)
reduced_growth_preds = tf.cast(reduced_growth_proba >= 0.5, tf.int32)
reduced_balanced_preds = tf.cast(reduced_balanced_proba >= 0.5, tf.int32)
reduced_conservative_preds = tf.cast(reduced_conservative_proba >= 0.5, tf.int32)

reduced_preds = {'alternative': [reduced_alternative_proba, reduced_alternative_preds],
              'aggressive': [reduced_aggressive_proba, reduced_aggressive_preds],
              'growth': [reduced_growth_proba, reduced_growth_preds],
              'balanced': [reduced_balanced_proba, reduced_balanced_preds],
              'conservative': [reduced_conservative_proba, reduced_conservative_preds]
             }

7/7 [==============================] - 0s 1ms/step


In [99]:
reduced_results = pd.DataFrame()
for p in portfolios:
    y_test = datafiles[p][3]
    proba = reduced_preds[p][0]
    preds = reduced_preds[p][1]
    roc = roc_auc_score(y_test,proba)
    f1 = f1_score(y_test, preds)
    df = pd.DataFrame()
    df.loc['roc_auc_score','reduced'] = roc
    df.loc['f1_score', 'reduced'] = f1
    df['type'] = p
    reduced_results = pd.concat([reduced_results, df], axis=0)


reduced_results.set_index('type', append=True, inplace=True)

reduced_results = reduced_results.reorder_levels(['type',0])

reduced_results

reduced
type                                
conservative roc_auc_score  0.495446
             f1_score       0.484848
balanced     roc_auc_score  0.503922
             f1_score       0.500000
growth       roc_auc_score  0.563332
             f1_score       0.532967
aggressive   roc_auc_score  0.534421
             f1_score       0.611227
alternative  roc_auc_score  0.548462
             f1_score       0.658228

In [100]:
full_results = pd.DataFrame()
for p in portfolios:
    y_test = datafiles[p][3]
    proba = full_preds[p][0]
    preds = full_preds[p][1]
    roc = roc_auc_score(y_test,proba)
    f1 = f1_score(y_test, preds)
    df = pd.DataFrame()
    df.loc['roc_auc_score','full'] = roc
    df.loc['f1_score', 'full'] = f1
    df['type'] = p
    full_results = pd.concat([full_results, df], axis=0)


full_results.set_index('type', append=True, inplace=True)

full_results = full_results.reorder_levels(['type',0])
full_results

full
type                                
conservative roc_auc_score  0.473393
             f1_score       0.442105
balanced     roc_auc_score  0.512834
             f1_score       0.466019
growth       roc_auc_score  0.514452
             f1_score       0.615385
aggressive   roc_auc_score  0.545075
             f1_score       0.530055
alternative  roc_auc_score  0.519740
             f1_score       0.619835

In [101]:
all_results = pd.concat([full_results, reduced_results], axis=1)

In [102]:
all_results

full   reduced
type                                          
conservative roc_auc_score  0.473393  0.495446
             f1_score       0.442105  0.484848
balanced     roc_auc_score  0.512834  0.503922
             f1_score       0.466019  0.500000
growth       roc_auc_score  0.514452  0.563332
             f1_score       0.615385  0.532967
aggressive   roc_auc_score  0.545075  0.534421
             f1_score       0.530055  0.611227
alternative  roc_auc_score  0.519740  0.548462
             f1_score       0.619835  0.658228

In [103]:
all_results.style.highlight_max(color='lightblue', axis = 1).set_caption("Metrics Comparison for All Models")

In [107]:
best = all_results[['reduced']]

In [108]:
best.reset_index(inplace=True)

In [110]:
best.to_csv(Path("./model_metrics/nn.csv"), index=False)